In [1]:

from pystac_client import Client  
from collections import defaultdict    
import json
import pandas as pd
import geopandas as gpd
import datetime
from collections import namedtuple
from osgeo import gdal
import rasterio
from rasterio.features import bounds
from pyproj import Transformer
import numpy as np


from typing import Dict


# Set up VSIcurl settings
gdal.SetConfigOption("GDAL_HTTP_COOKIEFILE", "~/cookies.txt")
gdal.SetConfigOption("GDAL_HTTP_COOKIEJAR", "~/cookies.txt")
gdal.SetConfigOption("GDAL_DISABLE_READDIR_ON_OPEN", "YES")
gdal.SetConfigOption("CPL_VSIL_CURL_ALLOWED_EXTENSIONS", "TIF")


In [2]:
STAC_URL = "https://cmr.earthdata.nasa.gov/stac"
GEOJSON_PATH = "./data/farm.geojson"
COLLECTIONS = ['HLSL30.v2.0', 'HLSS30.v2.0']
date_range = "2022-06/2022-06"

In [3]:
provider_client = Client.open(STAC_URL)

providers = [p for p in provider_client.get_children()]

for count, provider in enumerate(providers):
    print(f'{count} - {provider.title}')

0 - LARC_ASDC
1 - USGS_EROS
2 - ESA
3 - GHRC
4 - LAADS
5 - OBPG
6 - OB_DAAC
7 - ECHO
8 - ISRO
9 - LPCUMULUS
10 - EDF_DEV04
11 - GES_DISC
12 - ASF
13 - OMINRT
14 - EUMETSAT
15 - NCCS
16 - NSIDCV0
17 - PODAAC
18 - LARC
19 - USGS
20 - SCIOPS
21 - LANCEMODIS
22 - CDDIS
23 - JAXA
24 - AU_AADC
25 - ECHO10_OPS
26 - LPDAAC_ECS
27 - NSIDC_ECS
28 - ORNL_DAAC
29 - LM_FIRMS
30 - SEDAC
31 - LANCEAMSR2
32 - NOAA_NCEI
33 - USGS_LTA
34 - GESDISCCLD
35 - GHRSSTCWIC
36 - LARC_CLOUD
37 - ASIPS
38 - ESDIS
39 - POCLOUD
40 - NSIDC_CPRD
41 - ORNL_CLOUD
42 - FEDEO
43 - MLHUB
44 - XYZ_PROV
45 - GHRC_DAAC
46 - CSDA
47 - NRSCC
48 - CEOS_EXTRA
49 - AMD_KOPRI
50 - AMD_USAPDC
51 - MOPITT
52 - GHRC_CLOUD
53 - LPCLOUD
54 - CCMEO


In [4]:
with open(GEOJSON_PATH, "r") as fp:
    geojson = json.load(fp)

bbox = geojson["features"][0]["geometry"]
print(json.dumps(bbox, indent=2))

{
  "coordinates": [
    [
      [
        -92.81542340855101,
        45.44080475249953
      ],
      [
        -92.81538233065694,
        45.440285951753424
      ],
      [
        -92.81530017486931,
        45.43957403407592
      ],
      [
        -92.81433073657263,
        45.439553858140044
      ],
      [
        -92.8140103290003,
        45.43936362753681
      ],
      [
        -92.81270405197334,
        45.43933480466197
      ],
      [
        -92.81235078208567,
        45.440798988073425
      ],
      [
        -92.81542340855101,
        45.44080475249953
      ]
    ]
  ],
  "type": "Polygon"
}


In [5]:
catalog = Client.open(f'{STAC_URL}/LPCLOUD/')
products = [c for c in catalog.get_children()]
for p in products: 
    print(f"{p.id}: {p.title}")

ASTGTM.v003: ASTER Global Digital Elevation Model V003
HLSL30.v2.0: HLS Landsat Operational Land Imager Surface Reflectance and TOA Brightness Daily Global 30m v2.0
HLSS30.v2.0: HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance Daily Global 30m v2.0
MYD11_L2.v061: MODIS/Aqua Land Surface Temperature/Emissivity 5-Min L2 Swath 1km V061
MYD11A2.v061: MODIS/Aqua Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061
MYD11A1.v061: MODIS/Aqua Land Surface Temperature/Emissivity Daily L3 Global 1km SIN Grid V061
MYD09Q1.v061: MODIS/Aqua Surface Reflectance 8-Day L3 Global 250m SIN Grid V061
MYD09A1.v061: MODIS/Aqua Surface Reflectance 8-Day L3 Global 500m SIN Grid V061
MYD09GA.v061: MODIS/Aqua Surface Reflectance Daily L2G Global 1km and 500m SIN Grid V061
MYD09GQ.v061: MODIS/Aqua Surface Reflectance Daily L2G Global 250m SIN Grid V061


In [6]:
search = catalog.search(
    collections=COLLECTIONS,
    intersects=bbox,
    datetime=date_range,
    limit=100
)
print(search.matched())
item_collection = search.get_all_items()
items = item_collection.to_dict()["features"]


17


In [7]:
geometry = geojson
outpath = "./data_out"

def downloader(items, geometry, outpath):
    for idx, item in enumerate(items):
        for band, file_dict in item["assets"].items():
            fp = file_dict["href"]
            with rasterio.open(fp) as geo_fp:
                bbox = bounds(geometry)
                coord_transformer = Transformer.from_crs("epsg:4326", geo_fp.crs)
                # calculate pixels to be streamed in cog
                coord_upper_left = coord_transformer.transform(bbox[3], bbox[0])
                coord_lower_right = coord_transformer.transform(bbox[1], bbox[2])
                pixel_upper_left = geo_fp.index(coord_upper_left[0], coord_upper_left[1])
                pixel_lower_right = geo_fp.index(coord_lower_right[0], coord_lower_right[1])

                for pixel in pixel_upper_left + pixel_lower_right:
                    # If the pixel value is below 0, that means that
                    # the bounds are not inside of our available dataset.
                    if pixel < 0:
                        print("Provided geometry extends available datafile.")
                        print("Provide a smaller area of interest to get a result.")
                        exit()

                # make http range request only for bytes in window
                window = rasterio.windows.Window.from_slices(
                    (pixel_upper_left[0], pixel_lower_right[0]),
                    (pixel_upper_left[1], pixel_lower_right[1]),
                )
                subset = geo_fp.read(1, window=window)
                if outpath is not None:
                    height, width = subset.shape

                    profile = geo_fp.profile.copy()
                    profile["width"] = width
                    profile["height"] = height
                    transform = profile["transform"]
                    new_transform = rasterio.Affine(
                        transform[0],
                        transform[1],
                        coord_upper_left[0],
                        transform[3],
                        transform[4],
                        coord_upper_left[1],
                    )
                    profile["transform"] = new_transform
                    print(f"Writing {os.path.join(outpath, fp.split(r'/')[-1])}")
                    with rasterio.open(
                        os.path.join(outpath, fp.split("/")[-1]), "w", **profile
                    ) as dst:
                        dst.write(subset, 1)



In [8]:
""" Uncomment to run downloader
for item in items:
    downloader(item, geojson, outpath)
"""

' Uncomment to run downloader\nfor item in items:\n    downloader(item, geojson, outpath)\n'